In [1]:
import httpx
import polars as pl

In [2]:
INE_BASE_URL = "https://servicios.ine.es/wstempus/js/ES"

client = httpx.Client(
    base_url=INE_BASE_URL,
    limits=httpx.Limits(max_keepalive_connections=20),
    transport=httpx.HTTPTransport(retries=3),
)

In [3]:
def ine_request(client: httpx.Client, endpoint, paginate=True):
    page = 1
    data = []

    while True:
        params = {"det": 10}

        if paginate:
            params["page"] = page

        response = client.get(
            f"/{endpoint}", params=params, follow_redirects=True
        ).json()

        if not response:
            break

        data.extend(response)

        if len(response) < 500 or not paginate:
            break

        page += 1

    return data

In [4]:
# https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES

operaciones_disponibles = pl.DataFrame(ine_request(client, "OPERACIONES_DISPONIBLES"))
print(operaciones_disponibles.shape)
operaciones_disponibles.sample(5)

(108, 6)


Id,Cod_IOE,Nombre,Codigo,Referencia,Url
i64,str,str,str,list[struct[3]],str
42,"""30052""","""Índices de Cifras de Negocios …","""ICN""","[{23,""Índices de cifras de negocios en la industria"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736148782&idp=1254735576715""}]",null
66,"""30463""","""Estadística de Nulidades, Sepa…","""ENSD""","[{166,""Estadística de nulidades, separaciones y divorcios"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176798&idp=1254735573206""}]",null
406,"""30324""","""Estimacion del numero de defun…","""EDES""",null,"""https://www.ine.es/experimenta…"
197,"""30271""","""Tablas de Mortalidad""","""TM""","[{127,""Tablas de mortalidad"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177004&idp=1254735573002""}]",null
215,"""30468""","""Estadística de Violencia Domés…","""VGD""","[{171,""Estadística de violencia doméstica y violencia de género"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176866&idp=1254735573206""}]",null


In [5]:
# https://servicios.ine.es/wstempus/js/ES/VARIABLES?det=10

variables = pl.DataFrame(ine_request(client, "VARIABLES"))
variables.sample(5)

Id,Nombre,Codigo
i64,str,str
740,"""Tipo de matrimonio según sexo""",""""""
729,"""Lugar de nacimiento del cónyug…",""""""
956,"""SAU y Otras tierras""",""""""
1042,"""Año de llegada a Municipio""",""""""
774,"""Tamaño del municipio de nacimi…",""""""


In [6]:
t = []

for row in operaciones_disponibles.rows(named=True):
    tablas_operacion_url = f"TABLAS_OPERACION/{row["Id"]}"
    tablas_operacion = ine_request(client, tablas_operacion_url)

    t.extend(tablas_operacion)

In [7]:
tablas = pl.json_normalize(t)
print(tablas.shape)
tablas.sample(5)

(4941, 50)


Id,Nombre,Codigo,Anyo_Periodo_ini,FechaRef_fin,Ultima_Modificacion,Periodicidad.Id,Periodicidad.Nombre,Periodicidad.Codigo,Publicacion.Id,Publicacion.Nombre,Publicacion.Periodicidad.Id,Publicacion.Periodicidad.Nombre,Publicacion.Periodicidad.Codigo,Publicacion.Operacion,Publicacion.PubFechaAct.Id,Publicacion.PubFechaAct.Nombre,Publicacion.PubFechaAct.Fecha,Publicacion.PubFechaAct.Periodo.Id,Publicacion.PubFechaAct.Periodo.Valor,Publicacion.PubFechaAct.Periodo.Periodicidad.Id,Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre,Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo,Publicacion.PubFechaAct.Periodo.Dia_inicio,Publicacion.PubFechaAct.Periodo.Mes_inicio,Publicacion.PubFechaAct.Periodo.Codigo,Publicacion.PubFechaAct.Periodo.Nombre,Publicacion.PubFechaAct.Periodo.Nombre_largo,Publicacion.PubFechaAct.Anyo,Periodo_ini.Id,Periodo_ini.Valor,Periodo_ini.Periodicidad.Id,Periodo_ini.Periodicidad.Nombre,Periodo_ini.Periodicidad.Codigo,Periodo_ini.Dia_inicio,Periodo_ini.Mes_inicio,Periodo_ini.Codigo,Periodo_ini.Nombre,Periodo_ini.Nombre_largo,Anyo_Periodo_fin,Periodo_fin.Id,Periodo_fin.Valor,Periodo_fin.Periodicidad.Id,Periodo_fin.Periodicidad.Nombre,Periodo_fin.Periodicidad.Codigo,Periodo_fin.Dia_inicio,Periodo_fin.Mes_inicio,Periodo_fin.Codigo,Periodo_fin.Nombre,Periodo_fin.Nombre_largo
i64,str,str,str,str,i64,i64,str,str,i64,str,i64,str,str,list[struct[5]],i64,str,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
49126,"""Relaciones de las víctimas con…","""NAC""","""2011""","""null""",1715936400000,12,"""Anual""","""A""",312,"""Estadística de Violencia Domés…",12,"""Anual""","""A""","[{215,""30468"",""Estadística de Violencia Doméstica y Violencia de Género"",""VGD"",[{171,""Estadística de violencia doméstica y violencia de género"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176866&idp=1254735573206""}]}]",10605,"""Estadística de Violencia Domés…",1715936400000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null
36835,"""Medias y percentiles por sexo …","""NAC""","""2022""","""null""",1727082000000,105,"""Cuatrienal""","""C""",87,"""Encuesta de Estructura Salaria…",105,"""Cuatrienal""","""C""","[{121,""30133"",""Encuesta Cuatrienal de Estructura Salarial"",""EAES:Q"",[{64,""Encuestas de estructura salarial"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177025&idp=1254735976596""}]}]",11807,"""Encuesta de Estructura Salaria…",1727247600000,451,1,105,"""Cuatrienal""","""C""","""1""","""1""","""1""",null,"""Año""",2022,451,1,105,"""Cuatrienal""","""C""","""1""","""1""","""1""",null,"""Año""",null,null,null,null,null,null,null,null,null,null,null
25177,"""Gasto según sexo y edad del su…","""ECOICOP_Censo2011_NAC""","""2006""","""null""",1719478800000,12,"""Anual""","""A""",381,"""Encuesta de Presupuestos Famil…",12,"""Anual""","""A""","[{314,""30458"",""Encuesta de Presupuestos Familiares (EPF)"",""EPF"",[{163,""Encuesta de presupuestos familiares. Base 2006"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176806&idp=1254735976608""}]}]",10604,"""Encuesta de Presupuestos Famil…",1719478800000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null
65255,"""Inactivos por estado civil, se…","""NAC""","""2002""","""null""",1721977200000,3,"""Trimestral""","""Q""",330,"""Encuesta de Población Activa""",3,"""Trimestral""","""Q""","[{293,""30308"",""Encuesta de Población Activa (EPA)"",""EPA"",[{138,""Encuesta de población activa"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&idp=1254735976595""}]}]",10660,"""Encuesta de Población Activa T…",1721977200000,20,2,3,"""Trimestral""","""Q""","""1""",

In [8]:
def get_series_tabla_url(tabla_id):
    return (
        f"https://servicios.ine.es/wstempus/jsCache/ES/SERIES_TABLA/{tabla_id}?det=10"
    )


def get_tablas_download_url(tabla_id):
    return f"https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{tabla_id}.csv"


tablas = tablas.with_columns(
    pl.col("Id")
    .map_elements(get_series_tabla_url, return_dtype=pl.String)
    .alias("series_tabla_url"),
    pl.col("Id")
    .map_elements(get_tablas_download_url, return_dtype=pl.String)
    .alias("tablas_download_url"),
)

In [9]:
with open("series.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['series_tabla_url']}\n")
        f.write(f"\tout={t['Id']}.json\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

with open("tablas.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['tablas_download_url']}\n")
        f.write(f"\tout={t['Id']}.csv\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

In [10]:
import subprocess

subprocess.run(
    [
        "aria2c",
        "-i",
        "series.input.spec",
        "-j",
        "20",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ]
)

In [ ]:
subprocess.run(
    [
        "aria2c",
        "-i",
        "tablas.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ]
)

In [ ]:
import glob

from tqdm import tqdm

csv_files = glob.glob("dataset/tablas/*/*.csv")

for file in tqdm(csv_files):
    filename = file.split(".")[-2].split("/")[-1]

    (
        pl.scan_csv(
            file,
            separator=";",
            ignore_errors=True,
            truncate_ragged_lines=True,
        ).sink_parquet(
            f"dataset/tablas/{filename}/{filename}.parquet",
            compression="zstd",
            row_group_size=1024**2,
            type_coercion=True,
        )
    )

In [36]:
import os

for file in csv_files:
    os.remove(file)

In [39]:
from huggingface_hub import HfApi

In [40]:
api = HfApi(token=os.getenv("HUGGINGFACE_TOKEN"))

In [42]:
api.upload_large_folder(
    folder_path="dataset", repo_id="davidgasquez/ine", repo_type="dataset"
)